# Event-based Simulation with SimPy

In [85]:
import simpy as sp
import numpy as np
from typing import List, Tuple

# alternatively to using np.random's distributions,
#   you can use a distribution directly from scipy.stats:
# import scipy.stats

## SimPy Basics

SimPy has a few basic concepts to let you perform event-based simulation:
1. **Environment:** think of this as "the world" in which your simulation occurs
2. **Event:** an event that occurs in the environment -- it can be a happening, from time ticking to an action occurring
3. **Process:** described by Python generator functions -- they create events and `yield` (yeet) them out to the world
4. **Timeout:** a special time-based event, that describes how long it takes for an event to occur (assumed units)
5. **Resource:** an object representing a resource that can be acquired or released by Processes (gas station with limited fuel pumps)
6. **Container:** an object representing a resource that can be produced or consumed (like electrical power or apples)
7. **Store:** an object representing a store that can replenish or sell its items (dynamic quantities)

One thing that is key about event-based simulation is that, much like embedded systems, all processes are **interruptible** by priority, meaning that processes can be at different priorities (just like real life).

If you need to yield multiple events (actions) at a time, use the `&` operator.  If you need to yield at least one event (whichever event is closest), use the `|` operator.

### DMV customer example

Just to show everything in context, I'll modify the [Bank example](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html) to the DMV:

In [2]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 10  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 10  # Min. customer patience
MAX_PATIENCE = 40  # Max. customer patience


def source(env, number, interval, counter, the_rng):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer %02d' % i, counter, 12.0, the_rng)
        env.process(c)
        t = the_rng.exponential(interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank, the_rng):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('at time %7.2f, %s arrived' % (arrive, name))

    with counter.request() as req:
        patience = the_rng.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('at time %7.2f, %s waited %6.2f' % (env.now, name, wait))

            tib = the_rng.exponential(time_in_bank)
            yield env.timeout(tib)
            print('at time %7.2f, %s finished their transactions' % (env.now, name))

        else:
            # We reneged
            print('at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))


# Setup and start the simulation
print('SIMULATION: DMV in all of its glory (time units are minutes)')
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()

# Start processes and run
counter = sp.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, the_rng))
env.run()

SIMULATION: DMV in all of its glory (time units are minutes)
at time    0.00, Customer 00 arrived
at time    0.00, Customer 00 waited   0.00
at time   24.04, Customer 01 arrived
at time   26.84, Customer 02 arrived
at time   28.62, Customer 00 finished their transactions
at time   28.62, Customer 01 waited   4.58
at time   41.37, Customer 03 arrived
at time   42.16, Customer 04 arrived
at time   42.86, Customer 05 arrived
at time   59.67, Customer 02 left the line after waiting  32.83 minutes
at time   60.18, Customer 06 arrived
at time   64.88, Customer 03 left the line after waiting  23.51 minutes
at time   66.11, Customer 01 finished their transactions
at time   66.11, Customer 04 waited  23.95
at time   67.21, Customer 04 finished their transactions
at time   67.21, Customer 05 waited  24.34
at time   70.99, Customer 05 finished their transactions
at time   70.99, Customer 06 waited  10.81
at time   72.49, Customer 07 arrived
at time   76.62, Customer 08 arrived
at time   78.86, Cu

### Sample SimPy problem

Let's perform a simulation of four electric cars (EV, electric vehicles) trying to drive to and use a battery charging station. A car will perform three actions:
1. Driving to the EV charging station (accept the driving time as an input parameter).
2. Request/acquire a charging spot.
3. Charge the battery (accept the time it takes to charge the battery as an input parameter).

Define the charging station as only having two spots, so we have a likely chance of at least one car waiting for charging to complete.
Let's use a uniform distribution between 2 and 6 miles to model the distance each car has to drive in order to reach the EV charging station. Let's also assume that the cars can drive 35 miles per hour to reach the charging station.  Let's also use a uniform distribution to model the number of minutes that each car has to charge, from 30 minutes to 90 minutes.

LPT: Don't forget the random seed!

In [3]:
DISTANCE_MIN = 2  # miles
DISTANCE_MAX = 6  # miles
NUM_OF_CARS = 8  # cars
DRIVING_SPEED = 35  # miles per hour
CHARGING_MIN = 30  # mins
CHARGING_MAX = 90  # mins
CHARGING_SPOTS = 3
RANDOM_SEED = 7

In [4]:
def create_cars(env: sp.Environment, charging_station: sp.Resource,
                rng: np.random.Generator,
                number_of_cars: int, driving_distance_min: float,
                driving_distance_max: float, driving_speed: float,
                car_charging_min: float, car_charging_max: float):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed  # convert hours to minutes
        env.process(electric_car(env, charging_station, car_id, driving_time, charging_time))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car(env: sp.Environment, charging_station: sp.Resource,
                 name: str, driving_time: float, charging_time: float):
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f} and t_charge {charging_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)  # 1. driving to the station
    
    with charging_station.request() as charging_spot:  # 2a. request/acquire a charging spot
        wait_start = env.now
        print(f"At {env.now:.2f}, Car {name} is waiting for an EV charging spot.")
        yield charging_spot  # 2b. wait for an available EV charging spot
        
        wait_completed = env.now
        wait_duration = wait_completed - wait_start
        print(f"At {env.now:.2f}, Car {name} has acquired an EV charging spot after waiting {wait_duration:.2f}.")
        yield env.timeout(charging_time)  # 3. charge the battery
        
    print(f"At {env.now:.2f}, Car {name} has finished charging and has left.")

In [5]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.Resource(the_env, capacity=CHARGING_SPOTS)
create_cars(the_env, the_charging_station, the_rng, NUM_OF_CARS,
            DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
            CHARGING_MIN, CHARGING_MAX)
the_env.run()

At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58 and t_charge 67.51 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 4.97 and t_charge 76.54 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 9.42 and t_charge 48.01 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 9.06 and t_charge 30.32 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 0.00, Car 4 with t_drive 6.64 and t_charge 77.82 has spawned.
At 0.00, Car 4 is driving to the EV charging station.
At 0.00, Car 5 with t_drive 5.34 and t_charge 48.18 has spawned.
At 0.00, Car 5 is driving to the EV charging station.
At 0.00, Car 6 with t_drive 6.48 and t_charge 45.29 has spawned.
At 0.00, Car 6 is driving to the EV charging station.
At 0.00, Car 7 with t_drive 7.22 and t_charge 60.27 has spawned.
At 0.00, Car 7 is driving to the EV charging statio

## Handling priorities and interrupts

### Using PriorityResource
Let's bump the EV charging simulation up now, to 10 cars, and pretend that a couple of the cars have made reservations online, so they can jump the queue ([`simpy.PriorityResource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#priorityresource)).

In [6]:
DISTANCE_MIN = 2  # miles
DISTANCE_MAX = 6  # miles
NUM_OF_CARS = 10  # cars
DRIVING_SPEED = 35  # miles per hour
CHARGING_MIN = 30  # mins
CHARGING_MAX = 90  # mins
CHARGING_SPOTS = 3
RANDOM_SEED = 7

In [7]:
def create_cars_with_priorities(env: sp.Environment, charging_station: sp.Resource,
                rng: np.random.Generator,
                number_of_cars: int, driving_distance_min: float,
                driving_distance_max: float, driving_speed: float,
                car_charging_min: float, car_charging_max: float):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(rng.beta(5, 3)).astype(int)
        env.process(electric_car_with_priorities(env, charging_station, car_id,
                                                 driving_time, charging_time, 
                                                 priority))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car_with_priorities(env: sp.Environment, charging_station: sp.PriorityResource,
                                 name: str, driving_time: float, charging_time: float,
                                 charging_station_priority: int):
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f} and t_charge {charging_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)  # 1. driving to the station
    
    with charging_station.request(priority=charging_station_priority) as charging_spot:
        wait_start = env.now
        print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
              f"is waiting for an EV charging spot.")
        yield charging_spot  # 2b. wait for an available EV charging spot
        
        wait_completed = env.now
        wait_duration = wait_completed - wait_start
        print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
              f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
        yield env.timeout(charging_time)  # 3. charge the battery
        
    print(f"At {env.now:.2f}, Car {name} has finished charging and has left.")

In [8]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PriorityResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_priorities(the_env, the_charging_station, the_rng, NUM_OF_CARS,
                            DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
                            CHARGING_MIN, CHARGING_MAX)
the_env.run()

At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58 and t_charge 67.51 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 9.06 and t_charge 30.32 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 6.48 and t_charge 45.29 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 10.21 and t_charge 67.33 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 0.00, Car 4 with t_drive 6.96 and t_charge 32.14 has spawned.
At 0.00, Car 4 is driving to the EV charging station.
At 0.00, Car 5 with t_drive 4.75 and t_charge 30.71 has spawned.
At 0.00, Car 5 is driving to the EV charging station.
At 0.00, Car 6 with t_drive 4.49 and t_charge 79.80 has spawned.
At 0.00, Car 6 is driving to the EV charging station.
At 0.00, Car 7 with t_drive 8.52 and t_charge 68.38 has spawned.
At 0.00, Car 7 is driving to the EV charging stati

**Notice:** You can see that in the above output has cars with higher priority 0 that have "jumped the queue" over cars with lower priority 1.

### Using a time-based interrupt

Interrupts in a fair way, to indicate when a driver might have just gotten a text message and needs to leave and be somewhere instead.  We can simulate this by adding a "scheduled event" timeout in addition to the charging time timeout using the "|" (or) operator:

In [12]:
def create_cars_with_prio_leavetime(env: sp.Environment, charging_station: sp.Resource,
                                    rng: np.random.Generator,
                                    number_of_cars: int, driving_distance_min: float,
                                    driving_distance_max: float, driving_speed: float,
                                    car_charging_min: float, car_charging_max: float):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(rng.beta(5, 3)).astype(int)
        leave_time = rng.uniform(10., car_charging_max)
        env.process(electric_car_with_prio_leavetime(env, charging_station, car_id,
                                                     driving_time, charging_time, 
                                                     priority, leave_time))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car_with_prio_leavetime(env: sp.Environment, charging_station: sp.PriorityResource,
                                     name: str, driving_time: float, charging_time: float,
                                     charging_station_priority: int, leave_time: float):
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f} and t_charge {charging_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)  # 1. driving to the station
    
    with charging_station.request(priority=charging_station_priority) as charging_spot:
        wait_start = env.now
        print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
              f"is waiting for an EV charging spot.")
        yield charging_spot  # 2b. wait for an available EV charging spot
        
        wait_completed = env.now
        wait_duration = wait_completed - wait_start
        print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
              f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
        # either the charging or the leave timeout will trigger 
        #   first, use timeout_event to store which
        timeout_event = yield env.timeout(charging_time, value="charging_timeout") | env.timeout(leave_time, value="leave_timeout")
        
    print(f"At {env.now:.2f}, Car {name} has finished charging and has left due to {list(timeout_event.values())}.")

In [13]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PriorityResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_prio_leavetime(the_env, the_charging_station, the_rng, NUM_OF_CARS,
                                DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
                                CHARGING_MIN, CHARGING_MAX)
the_env.run()

At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58 and t_charge 67.51 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 8.89 and t_charge 79.27 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 7.22 and t_charge 60.27 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 3.67 and t_charge 32.64 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 0.00, Car 4 with t_drive 5.13 and t_charge 59.81 has spawned.
At 0.00, Car 4 is driving to the EV charging station.
At 0.00, Car 5 with t_drive 9.12 and t_charge 30.22 has spawned.
At 0.00, Car 5 is driving to the EV charging station.
At 0.00, Car 6 with t_drive 8.52 and t_charge 68.38 has spawned.
At 0.00, Car 6 is driving to the EV charging station.
At 0.00, Car 7 with t_drive 3.83 and t_charge 65.89 has spawned.
At 0.00, Car 7 is driving to the EV charging statio

**Notice:** Some of the cars above waited the whole charging time, others have just left because they need to go somewhere else.

### Using PreemptiveResource 

We could even say someone has a VIP card that can kick someone off of a pump ([`simpy.PreemptiveResource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#preemptiveresource)), though that would probably not be a fair system at all.  However, you can imagine there are simulation cases where you would want [preemption](https://en.wikipedia.org/wiki/Preemption_(computing)), such as in an embedded system that requires fast response times, networks that require resources to be shutdown for maintenance, etc. 

To catch when an interrupt occurs, we have to use a [try-except statement](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).  We also have to use a while loop to re-enter the line.  Let's see it in action: 

In [36]:
def create_cars_with_prio_leave_vip(env: sp.Environment, charging_station: sp.Resource,
                                    rng: np.random.Generator,
                                    number_of_cars: int, driving_distance_min: float,
                                    driving_distance_max: float, driving_speed: float,
                                    car_charging_min: float, car_charging_max: float):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(rng.beta(5, 3)).astype(int)
        leave_time = rng.uniform(10., car_charging_max)
        env.process(electric_car_with_prio_leave_vip(
            env, charging_station, car_id, driving_time,
            charging_time, priority, leave_time))
    print(f"At {env.now}, all cars have been added to the environment.")


def electric_car_with_prio_leave_vip(env: sp.Environment, charging_station: sp.PriorityResource,
                                     name: str, driving_time: float, charging_time: float,
                                     charging_station_priority: int, leave_time: float):
    # setup placeholders for data
    charge_left = charging_time  # store the charging time needed separately
    leave_left = leave_time  # store the leave time separately
    timeout_event = None  # store the timeout event for later use
    charging_timeout = None  # store the charging timeout for later use
    leave_timeout = None  # store the leave timeout for later use
    reenter_line = True  # state variable to keep track of whether to re-enter the line
    
    print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f}, "
          f"t_charge {charging_time:.2f}, and t_leave {leave_time:.2f} has spawned.")
    print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    yield env.timeout(driving_time)
    
    # while loop so we can re-insert this car into the queue automatically
    while reenter_line:
        with charging_station.request(priority=charging_station_priority) as charging_spot:
            wait_start = env.now
            leave_timeout = env.timeout(leave_left, value="leave_timeout")
            print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
                  f"is waiting for an EV charging spot.")
            timeout_event = yield charging_spot | leave_timeout
            
            if leave_timeout in timeout_event:
                print(f"At {env.now:.2f}, Car {name} gave up waiting in line...")
                return

            wait_completed = env.now
            wait_duration = wait_completed - wait_start
            print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
                  f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
            
            # We have to use a try-except-finally to handle the interrupt:
            try:
                charge_start = env.now
                charging_timeout = env.timeout(charge_left, value="charging_timeout")
                timeout_event = yield charging_timeout | leave_timeout
                reenter_line = False  # leave the line if we finished
            except sp.Interrupt as the_interrupt:
                interrupter = the_interrupt.cause.by
                print(f"At {env.now:.2f}, Car {name} was interrupted by {interrupter}. "
                      f"Getting back in line and waiting additional 10...")
                leave_left += 10.
            finally:
                # this lets us execute the computation in either case
                charge_duration = env.now - charge_start
                charge_left -= charge_duration
                leave_left -= charge_duration
    
    if charging_timeout in timeout_event:
        print(f"At {env.now:.2f}, Car {name} has finished charging and left.")
    elif leave_timeout in timeout_event:
        print(f"At {env.now:.2f}, Car {name} has to go somewhere and left with "
              f"{charge_left:.2f} charge required.")

In [37]:
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PreemptiveResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_prio_leave_vip(the_env, the_charging_station, the_rng, NUM_OF_CARS,
                                DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
                                CHARGING_MIN, CHARGING_MAX)
the_env.run()

At 0, all cars have been added to the environment.
At 0.00, Car 0 with t_drive 9.58, t_charge 67.51, and t_leave 10.42 has spawned.
At 0.00, Car 0 is driving to the EV charging station.
At 0.00, Car 1 with t_drive 8.89, t_charge 79.27, and t_leave 45.61 has spawned.
At 0.00, Car 1 is driving to the EV charging station.
At 0.00, Car 2 with t_drive 7.22, t_charge 60.27, and t_leave 59.00 has spawned.
At 0.00, Car 2 is driving to the EV charging station.
At 0.00, Car 3 with t_drive 3.67, t_charge 32.64, and t_leave 51.13 has spawned.
At 0.00, Car 3 is driving to the EV charging station.
At 0.00, Car 4 with t_drive 5.13, t_charge 59.81, and t_leave 39.56 has spawned.
At 0.00, Car 4 is driving to the EV charging station.
At 0.00, Car 5 with t_drive 9.12, t_charge 30.22, and t_leave 77.77 has spawned.
At 0.00, Car 5 is driving to the EV charging station.
At 0.00, Car 6 with t_drive 8.52, t_charge 68.38, and t_leave 38.90 has spawned.
At 0.00, Car 6 is driving to the EV charging station.
At 0

**Notice:** We can capture if a car gives up in line, but if we extended the leave timeout then you can also see interrupted cars wait it out.

## Monitoring your SimPy simulation

There are three ways to go about monitoring a simulation, in order from easy to difficult:
1. Implement your own monitoring harness (basically, add input/output data structures (such as lists) for monitoring into your process functions and classes)
2. Patching (creating a wrapper) for Simpy `Events` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing))
3. Patching (creating a wrapper) for SimPy `Resources` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#resource-usage))

So, we can go ahead and use a dictionary and some lists to basically make our monitoring harness.  If we want to be fancier and make it available for analysis immediately after, we can use a pandas DataFrame after.  Either way, we must think about how we want to store our data (the columns of the table, the type of information, the granularity).  Sometimes this is called a _data schema_.

So, let's go ahead and log the following information: Event time, Car ID, Car Priority, Drive Time, Available Time Left, Charge Time Left, Event

Let's see how to setup our previous code so that we can finalize it into a `pandas.DataFrame` so we can easily get analyze it:

In [93]:
def create_cars_with_prio_leave_vip(env: sp.Environment, charging_station: sp.Resource,
                                    rng: np.random.Generator,
                                    number_of_cars: int, driving_distance_min: float,
                                    driving_distance_max: float, driving_speed: float,
                                    car_charging_min: float, car_charging_max: float,
                                    list_logs: List[Tuple[float, int, int, float, float, float, str]]):
    for car_id in range(number_of_cars):
        charging_time = rng.uniform(car_charging_min, car_charging_max)
        driving_distance = rng.uniform(driving_distance_min, driving_distance_max)
        driving_time = 60 * driving_distance / driving_speed
        # use beta distribution below so most cars have
        #   lower priority (priority 1) than VIPs (priority 0)
        priority = np.round(rng.beta(5, 3)).astype(int)
        leave_time = rng.uniform(10., car_charging_max)
        env.process(electric_car_with_prio_leave_vip(
            env, charging_station, car_id, driving_time,
            charging_time, priority, leave_time,
            list_logs))
    print(f"At {env.now}, all cars have been added to the environment.")


def add_event_to_log(list_logs: List[Tuple[float, int, int, float, float, float, str]],
                     event_time: float, car_id: int, car_prio: int,
                     drive_time: float, leave_time_left: float,
                     charge_time_left: float, event: str):
    list_logs.append({'Event time': event_time,
                      'Car ID': car_id, 
                      'Car priority': car_prio, 
                      'Drive time': drive_time,
                      'Available time left': leave_time_left,
                      'Charge time left': charge_time_left,
                      'Event': event})


def electric_car_with_prio_leave_vip(env: sp.Environment, charging_station: sp.PriorityResource,
                                     name: str, driving_time: float, charging_time: float,
                                     charging_station_priority: int, leave_time: float,
                                     list_logs: List[Tuple[float, int, int, float, float, float, str]]):
    # setup placeholders for data
    charge_left = charging_time
    leave_left = leave_time
    timeout_event = None
    charging_timeout = None
    leave_timeout = None
    reenter_line = True
    
    # you can comment out all of the prints because we have a log now.
    # print(f"At {env.now:.2f}, Car {name} with t_drive {driving_time:.2f}, "
    #       f"t_charge {charging_time:.2f}, and t_leave {leave_time:.2f} has spawned.")
    add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                     leave_left, charge_left, "spawned")
    # print(f"At {env.now:.2f}, Car {name} is driving to the EV charging station.")
    add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                     leave_left, charge_left, "driving to charging station")
    yield env.timeout(driving_time)
    
    # while loop so we can re-insert this car into the queue automatically
    while reenter_line:
        with charging_station.request(priority=charging_station_priority) as charging_spot:
            wait_start = env.now
            leave_timeout = env.timeout(leave_left, value="leave_timeout")
            # print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
            #       f"is waiting for an EV charging spot.")
            add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                             leave_left, charge_left, "waiting for spot")
            timeout_event = yield charging_spot | leave_timeout
            
            if leave_timeout in timeout_event:
                # print(f"At {env.now:.2f}, Car {name} gave up waiting in line...")
                add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                                 leave_left, charge_left, "gave up on charging station line")
                return

            wait_completed = env.now
            wait_duration = wait_completed - wait_start
            # print(f"At {env.now:.2f}, Car {name} with priority {charging_station_priority} "
            #       f"has acquired an EV charging spot after waiting {wait_duration:.2f}.")
            # NOTE: unfortunately, the wait duration isn't calculated until
            #       later so we include the wait duration as an event, we can
            #       extract the wait duration afterwards.
            add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                             leave_left, charge_left, "acquired charging spot after waiting {wait_duration:.2f}")
            
            # We have to use a try-except-finally to handle the interrupt:
            try:
                charge_start = env.now
                charging_timeout = env.timeout(charge_left, value="charging_timeout")
                timeout_event = yield charging_timeout | leave_timeout
                reenter_line = False  # leave the line if we finished
            except sp.Interrupt as the_interrupt:
                interrupter = the_interrupt.cause.by
                # print(f"At {env.now:.2f}, Car {name} was interrupted by {interrupter}. "
                #       f"Getting back in line and waiting additional 10...")
                add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                                 leave_left, charge_left, "interrupted by vip")
                leave_left += 10.
            finally:
                # this lets us execute the computation in either case
                charge_duration = env.now - charge_start
                charge_left -= charge_duration
                leave_left -= charge_duration
    
    if charging_timeout in timeout_event:
        # print(f"At {env.now:.2f}, Car {name} has finished charging and left.")
        add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                         leave_left, charge_left, "finished charging and left")
    elif leave_timeout in timeout_event:
        # print(f"At {env.now:.2f}, Car {name} has to go somewhere and left with "
        #       f"{charge_left:.2f} charge required.")
        add_event_to_log(list_logs, env.now, name, charging_station_priority, driving_time, 
                         leave_left, charge_left, "left due to limit on available time")

In [94]:
the_logs = []
the_rng = np.random.default_rng(RANDOM_SEED)
the_env = sp.Environment()
the_charging_station = sp.PreemptiveResource(the_env, capacity=CHARGING_SPOTS)
create_cars_with_prio_leave_vip(the_env, the_charging_station, the_rng, NUM_OF_CARS,
                                DISTANCE_MIN, DISTANCE_MAX, DRIVING_SPEED,
                                CHARGING_MIN, CHARGING_MAX, the_logs)
the_env.run()

At 0, all cars have been added to the environment.


In [95]:
# check what's inside the logs
the_logs[:2]

[{'Event time': 0,
  'Car ID': 0,
  'Car priority': 1,
  'Drive time': 9.580894635219947,
  'Available time left': 10.421224365245978,
  'Charge time left': 67.50572799628002,
  'Event': 'spawned'},
 {'Event time': 0,
  'Car ID': 0,
  'Car priority': 1,
  'Drive time': 9.580894635219947,
  'Available time left': 10.421224365245978,
  'Charge time left': 67.50572799628002,
  'Event': 'driving to charging station'}]

In [97]:
import pandas as pd

df_simulation = pd.DataFrame(the_logs)
df_simulation.head()

,Event time,Car ID,Car priority,Drive time,Available time left,Charge time left,Event
0,0.0,0,1,9.580895,10.421224,67.505728,spawned
1,0.0,0,1,9.580895,10.421224,67.505728,driving to charging station
2,0.0,1,1,8.894190,45.606104,79.273705,spawned
3,0.0,1,1,8.894190,45.606104,79.273705,driving to charging station
4,0.0,2,1,7.223982,59.003168,60.272896,spawned


Now, say that we want to find out the average amount of charge time left on all cars.  One of the ways we can get this is to group by the Car ID, then get the minimum charge time of each car (or last time event of each car), and then take the mean of that:

In [104]:
mean_charge_time_starting = df_simulation.groupby(by="Car ID")["Charge time left"].max().mean()
std_charge_time_starting = df_simulation.groupby(by="Car ID")["Charge time left"].max().std()
mean_charge_time_left = df_simulation.groupby(by="Car ID")["Charge time left"].min().mean()
std_charge_time_left = df_simulation.groupby(by="Car ID")["Charge time left"].min().std()

print(f"The mean starting charge time left was {mean_charge_time_starting:.2f} +/- {std_charge_time_starting:.2f}.")
print(f"The mean final charge time left was {mean_charge_time_left:.2f} +/- {std_charge_time_left:.2f}.")

The mean starting charge time left was 59.71 +/- 18.96.
The mean final charge time left was 40.41 +/- 28.70.


We see that for some reason, there are certain cars that did not get any charge time at all, pretty much.  If we were trying to optimize for a fair system, then we'd want to tweak our settings (perhaps get rid of the VIP provision!) until the standard deviation is much lower.  This is the power of discrete-event simulation -- we are able to run stochastic simulations that otherwise may be difficult to compute deterministically.

# Using SimPy for real-time ECE problems

Turns out that SimPy also has a RealtimeEnvironment, which lets you synchronize events with wall-clock time.

This means that we can perform the following simulations:
1. Hardware-in-the-loop testing (VERY useful in electronics, embedded systems, medical devices, networking, etc.)
2. Testing that requires human interaction (energy, sustainability, etc.)

**Let's discuss:** what might be an event-based simulation problem in your specialization